# Comparison with Lee & Lee 2025

This notebook performs a direct comparison between AILS and the MCPP-GAK algorithm
from Lee & Lee (2025) using the same Moving AI Lab benchmarks.

**Reference:** Lee, C., & Lee, J. (2025). Multi-Agent Coverage Path Planning Using 
Graph-Adapted K-Means in Road Network Digital Twin. *Electronics*, 14(1), 89.

**Author:** Amr Elshahed  
**Institution:** Universiti Sains Malaysia

In [ ]:
import sys
sys.path.insert(0, '.')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

from AILS_complete import (
    AILSPathfinder, MovingAIMapLoader, run_benchmark
)

# Set style
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.family'] = 'serif'

print("Lee & Lee 2025 Comparison Notebook")
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Lee & Lee 2025 Reported Results

Results from Table 2 and Table 3 of the original paper.
These are for MCPP (Multi-agent Coverage Path Planning) with different agent counts.

In [ ]:
# Lee & Lee 2025 results (from paper Tables 2-3)
lee_lee_results = pd.DataFrame([
    # den312d
    {'map': 'den312d', 'k': 2, 'setting': 'arbitrary', 'max_length': 2249, 'time': 1226.56, 'iterations': 4},
    {'map': 'den312d', 'k': 2, 'setting': 'clutter', 'max_length': 4889, 'time': 599.77, 'iterations': 2},
    {'map': 'den312d', 'k': 5, 'setting': 'arbitrary', 'max_length': 1565, 'time': 504.49, 'iterations': 2},
    {'map': 'den312d', 'k': 5, 'setting': 'clutter', 'max_length': 3491, 'time': 572.03, 'iterations': 2},
    {'map': 'den312d', 'k': 10, 'setting': 'arbitrary', 'max_length': 623, 'time': 10746.30, 'iterations': 3},
    {'map': 'den312d', 'k': 10, 'setting': 'clutter', 'max_length': 3057, 'time': 1660.64, 'iterations': 2},
    {'map': 'den312d', 'k': 20, 'setting': 'arbitrary', 'max_length': 371, 'time': 19138.61, 'iterations': 2},
    {'map': 'den312d', 'k': 20, 'setting': 'clutter', 'max_length': 3357, 'time': 672.46, 'iterations': 2},
    
    # ht_chantry
    {'map': 'ht_chantry', 'k': 2, 'setting': 'arbitrary', 'max_length': 14921, 'time': 5979.80, 'iterations': 2},
    {'map': 'ht_chantry', 'k': 2, 'setting': 'clutter', 'max_length': 14921, 'time': 5957.24, 'iterations': 2},
    {'map': 'ht_chantry', 'k': 5, 'setting': 'arbitrary', 'max_length': 5081, 'time': 13221.50, 'iterations': 2},
    {'map': 'ht_chantry', 'k': 5, 'setting': 'clutter', 'max_length': 10065, 'time': 4808.83, 'iterations': 2},
    {'map': 'ht_chantry', 'k': 10, 'setting': 'arbitrary', 'max_length': 3601, 'time': 33464.44, 'iterations': 2},
    {'map': 'ht_chantry', 'k': 10, 'setting': 'clutter', 'max_length': 7379, 'time': 4608.55, 'iterations': 2},
    {'map': 'ht_chantry', 'k': 20, 'setting': 'arbitrary', 'max_length': 1847, 'time': 147405.96, 'iterations': 4},
    {'map': 'ht_chantry', 'k': 20, 'setting': 'clutter', 'max_length': 7567, 'time': 4620.24, 'iterations': 2},
    
    # random-64-64-20
    {'map': 'random-64-64-20', 'k': 2, 'setting': 'arbitrary', 'max_length': 6539, 'time': 1152.05, 'iterations': 2},
    {'map': 'random-64-64-20', 'k': 2, 'setting': 'clutter', 'max_length': 6539, 'time': 1082.83, 'iterations': 2},
    {'map': 'random-64-64-20', 'k': 5, 'setting': 'arbitrary', 'max_length': 1295, 'time': 6531.38, 'iterations': 3},
    {'map': 'random-64-64-20', 'k': 5, 'setting': 'clutter', 'max_length': 4099, 'time': 958.85, 'iterations': 2},
    {'map': 'random-64-64-20', 'k': 10, 'setting': 'arbitrary', 'max_length': 1309, 'time': 962.04, 'iterations': 2},
    {'map': 'random-64-64-20', 'k': 10, 'setting': 'clutter', 'max_length': 2833, 'time': 947.77, 'iterations': 2},
    {'map': 'random-64-64-20', 'k': 20, 'setting': 'arbitrary', 'max_length': 817, 'time': 34094.00, 'iterations': 3},
    {'map': 'random-64-64-20', 'k': 20, 'setting': 'clutter', 'max_length': 3785, 'time': 1025.53, 'iterations': 2},
])

print("Lee & Lee 2025 MCPP-GAK Results:")
print(lee_lee_results.to_string(index=False))

## 2. Run AILS on Same Benchmarks

In [ ]:
# Benchmark maps used in Lee & Lee 2025
BENCHMARK_MAPS = {
    'den312d': '../data/benchmark_maps/den312d.map',
    'ht_chantry': '../data/benchmark_maps/ht_chantry.map',
    'random-64-64-20': '../data/benchmark_maps/random-64-64-20.map'
}

NUM_SCENARIOS = 100  # Scenarios per map

ails_results = []

for map_name, map_path in BENCHMARK_MAPS.items():
    print(f"\nProcessing {map_name}...")
    
    if not os.path.exists(map_path):
        print(f"  Map not found: {map_path}")
        continue
    
    try:
        # Load map
        grid = MovingAIMapLoader.load_map(map_path)
        print(f"  Grid size: {grid.shape}")
        print(f"  Traversable: {np.sum(grid == 0)} cells")
        
        # Run AILS experiments
        results = run_benchmark(grid, NUM_SCENARIOS, seed=42)
        
        # Aggregate results
        for method, result_list in results.items():
            times = [r.time_ms for r in result_list if r.path_found]
            nodes = [r.nodes_visited for r in result_list if r.path_found]
            
            if times:
                ails_results.append({
                    'map': map_name,
                    'method': method,
                    'time_mean_ms': np.mean(times),
                    'time_std_ms': np.std(times),
                    'nodes_mean': np.mean(nodes),
                    'nodes_std': np.std(nodes),
                    'success_rate': len(times) / len(result_list) * 100,
                    'num_scenarios': len(result_list)
                })
                
    except Exception as e:
        print(f"  Error: {e}")

df_ails = pd.DataFrame(ails_results)
print("\n" + "="*70)
print("AILS Results on Lee & Lee 2025 Benchmarks:")
print(df_ails.to_string(index=False))

## 3. Comparative Analysis

In [ ]:
# Compare methodological differences
print("\n" + "="*70)
print("METHODOLOGICAL COMPARISON")
print("="*70)
print("""
| Aspect            | Lee & Lee 2025 (MCPP-GAK)    | AILS (This Work)           |
|-------------------|------------------------------|----------------------------|
| Problem Type      | Multi-agent coverage (MCPP)  | Single-agent pathfinding   |
| Algorithm Base    | Graph-Adapted K-Means        | Corridor-constrained A*    |
| Search Strategy   | Clustering + ILS             | Bresenham + Adaptive A*    |
| Optimality        | Heuristic (near-optimal)     | Optimal within corridor    |
| Agent Support     | Multi-agent (k=2-40)         | Single-agent               |
| Adaptivity        | Iterative balancing          | Local density-based        |
| Benchmarks        | Moving AI Lab                | Moving AI Lab (shared)     |
""")

In [ ]:
# Per-map comparison visualization
if len(df_ails) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    
    for idx, map_name in enumerate(BENCHMARK_MAPS.keys()):
        ax = axes[idx]
        
        # Get AILS results for this map
        df_map = df_ails[df_ails['map'] == map_name]
        
        if len(df_map) > 0:
            methods = df_map['method'].values
            times = df_map['time_mean_ms'].values
            stds = df_map['time_std_ms'].values
            
            x = range(len(methods))
            ax.bar(x, times, yerr=stds, capsize=5, alpha=0.7)
            ax.set_xticks(x)
            ax.set_xticklabels([m.replace('_', '\n') for m in methods], fontsize=8)
            ax.set_ylabel('Time (ms)')
            ax.set_title(map_name)
            ax.set_yscale('log')
    
    plt.suptitle('AILS Performance on Lee & Lee 2025 Benchmarks', y=1.02)
    plt.tight_layout()
    plt.savefig('../data/results/ails_lee_lee_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()

## 4. Node Reduction Analysis

In [ ]:
# Calculate node reduction compared to standard A*
if len(df_ails) > 0:
    print("\n" + "="*70)
    print("NODE REDUCTION ANALYSIS")
    print("="*70)
    
    for map_name in BENCHMARK_MAPS.keys():
        df_map = df_ails[df_ails['map'] == map_name]
        
        if len(df_map) == 0:
            continue
            
        std_nodes = df_map[df_map['method'] == 'standard_astar']['nodes_mean'].values
        
        if len(std_nodes) > 0:
            std_nodes = std_nodes[0]
            
            print(f"\n{map_name}:")
            print(f"  Standard A* nodes: {std_nodes:.1f}")
            
            for method in ['ails_base', 'ails_adaptive']:
                ails_nodes = df_map[df_map['method'] == method]['nodes_mean'].values
                if len(ails_nodes) > 0:
                    ails_nodes = ails_nodes[0]
                    reduction = (1 - ails_nodes / std_nodes) * 100
                    print(f"  {method}: {ails_nodes:.1f} nodes ({reduction:.1f}% reduction)")

## 5. Summary Table for Paper

In [ ]:
# Generate LaTeX table for paper
if len(df_ails) > 0:
    print("\n" + "="*70)
    print("LATEX TABLE FOR PAPER")
    print("="*70)
    
    latex = """\\begin{table}[ht]
\\centering
\\caption{AILS Performance on Lee \\& Lee 2025 Benchmark Maps}
\\label{tab:lee_comparison}
\\begin{tabular}{lcccc}
\\toprule
Map & Method & Time (ms) & Nodes & Success (\\%) \\\\
\\midrule
"""
    
    for _, row in df_ails.iterrows():
        method_short = row['method'].replace('standard_', 'std_').replace('ails_', 'ails-')
        latex += f"{row['map']} & {method_short} & {row['time_mean_ms']:.2f} & {row['nodes_mean']:.0f} & {row['success_rate']:.1f} \\\\ \n"
    
    latex += """\\bottomrule
\\end{tabular}
\\end{table}"""
    
    print(latex)
    
    # Save LaTeX table
    with open('../data/results/lee_comparison_table.tex', 'w') as f:
        f.write(latex)
    print("\nTable saved to ../data/results/lee_comparison_table.tex")

In [ ]:
print("\n" + "="*70)
print("Comparison analysis complete!")
print("="*70)